### 모델 학습 및 검증

In [4]:
!pip install tensorflow-addons

     |████████████████████████████████| 703 kB 1.5 MB/s eta 0:00:01


In [1]:
import glob
import os
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import Conv2D, MaxPool2D, Concatenate, Flatten, Dense

In [2]:
# 파라미터
EPOCHS = 1000
RESULT_SAVE_PATH = 'dataset/results/'

### Inception-base 모델정의

In [3]:
def Model():
    def inception(filters):
        def subnetwork(x):
            h1 = Conv2D(filters, (1, 1), padding='same', activation='relu')(x)
            h1 = MaxPool2D()(h1)
            
            h2 = Conv2D(filters // 2, (1, 1), padding='same', activation='relu')(x)
            h2 = Conv2D(filters, (3, 3), padding='same', activation='relu')(h2)
            h2 = MaxPool2D()(h2)
            
            h3 = Conv2D(filters // 2, (1, 1), padding='same', activation='relu')(x)
            h3 = Conv2D(filters, (5, 5), padding='same', activation='relu')(h3)
            h3 = MaxPool2D()(h3)
            return Concatenate()([h1, h2, h3])
        return subnetwork
    x = tf.keras.Input(shape=(256, 256, 3))
    h = inception(16)(x)
    h = inception(32)(h)
    h = inception(32)(h)
    h = inception(32)(h)
    h = inception(32)(h)
    h = Flatten()(h)
    h = Dense(1024, activation='relu')(h)
    y = Dense(1, activation='sigmoid')(h)
    return tf.keras.Model(inputs=x, outputs=y)

In [4]:
# 데이타 전처리 함수
def preprocess(img):
    return tf.image.convert_image_dtype(img, tf.float32)

### Data Augmentation 정의

In [5]:
def augmentation(img, label):
    def flip(x):
        x = tf.image.random_flip_left_right(x)
        x = tf.image.random_flip_up_down(x)
        return x
    def rotate(x):
        x = tf.cond(tf.random.uniform(shape=[], minval=0.0, maxval=1.0, dtype=tf.float32) > 0.5,
                   lambda: tfa.image.rotate(x, 
                                            tf.random.uniform(shape=[], minval=0.0, maxval=360.0, dtype=tf.float32),
                                            interpolation='BILINEAR'), 
                    lambda: x)
        return x
    def translation(x):
        dx = tf.random.uniform(shape=[], minval=-10.0, maxval=10.0, dtype=tf.float32)
        dy = tf.random.uniform(shape=[], minval=-10.0, maxval=10.0, dtype=tf.float32)
        x = tf.cond(tf.random.uniform(shape=[], minval=0.0, maxval=1.0, dtype=tf.float32) > 0.5,
                   lambda: tfa.image.transform(x, 
                                               [0, 0, dx, 0, 0, dy, 0, 0], 
                                               interpolation='BILINEAR'),
                   lambda: x)
        return x
    img = flip(img)
    img = rotate(img)
    img = translation(img)
    return img, label

## TFRecords 불러오기

In [6]:
tffiles = glob.glob('dataset/tfrecords/*')
raw_image_dataset = tf.data.TFRecordDataset(tffiles)

In [7]:
tffiles[1]

'dataset/tfrecords/00013.tfrecords'

In [8]:
image_feature_description = {
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'depth': tf.io.FixedLenFeature([], tf.int64),
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, image_feature_description)
def _parse_image_label(parsed_dataset):
    return preprocess(tf.image.decode_png(parsed_dataset['image_raw'])), parsed_dataset['label']

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
dataset = parsed_image_dataset.map(_parse_image_label)

### 데이타셋 나누기

In [9]:
ds_size = 0
for _ in dataset:
    ds_size += 1
    
train_size = int(ds_size * 0.7)
ds = dataset.shuffle(ds_size)
ds_train = ds.take(train_size).shuffle(1024, reshuffle_each_iteration=True).prefetch(1024).batch(32).map(augmentation)
ds_valid = ds.skip(train_size).prefetch(1024).batch(32)

## 모델 생성

In [10]:
model = Model()

In [ ]:
model.summary()

In [11]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=1)
history = model.fit(ds_train, validation_data=ds_valid, epochs=EPOCHS, callbacks=[earlystopping])

Epoch 1/1000
55/55 [==============================] - 300s 3s/step - loss: 0.7224 - accuracy: 0.5047 - val_loss: 0.6801 - val_accuracy: 0.4462
Epoch 2/1000
55/55 [==============================] - 26s 333ms/step - loss: 0.6978 - accuracy: 0.4812 - val_loss: 0.6896 - val_accuracy: 0.5632
Epoch 3/1000
55/55 [==============================] - 25s 340ms/step - loss: 0.6952 - accuracy: 0.4789 - val_loss: 0.6913 - val_accuracy: 0.4758
Epoch 4/1000
55/55 [==============================] - 27s 344ms/step - loss: 0.6913 - accuracy: 0.5212 - val_loss: 0.6990 - val_accuracy: 0.4879
Epoch 5/1000
55/55 [==============================] - 25s 338ms/step - loss: 0.6943 - accuracy: 0.4949 - val_loss: 0.6921 - val_accuracy: 0.5121
Epoch 6/1000
55/55 [==============================] - 23s 317ms/step - loss: 0.6921 - accuracy: 0.5028 - val_loss: 0.6999 - val_accuracy: 0.9086
Epoch 7/1000
55/55 [==============================] - 26s 334ms/step - loss: 0.6946 - accuracy: 0.4888 - val_loss: 0.6811 - val_accu

### 학습 결과 확인

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure()
plt.plot(loss, 'ro-')
plt.plot(val_loss, 'bo-')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch')
plt.title('Training and Validation Loss')
plt.show()

## 모델 저장

In [ ]:
model.save('dataset/inception_model.h5')